In [1]:
%pip install spuco

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
# set torch device to use metal
# set to cuda if needed
DEVICE = torch.device("mps")

In [3]:
# initialize seed
from spuco.utils import set_seed
set_seed(0)

In [4]:
# -------------------------
# STEP 1: Train with ERM
# -------------------------

from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

# MNIST classes and spurious feature difficulty setting
classes = [[0,1], [2,3], [4,5], [6,7], [8,9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

# initialize validation set from SpuCoMNIST
valset = SpuCoMNIST(
    root="data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes = classes,
    split="val",
    download=True
)
valset.initialize()

100%|██████████| 11996/11996 [00:00<00:00, 30666.43it/s]


In [5]:
from spuco.robust_train import ERM
import torchvision.transforms as T

# initialize training set from SpuCoMNIST
trainset = SpuCoMNIST(
    root="data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes = classes,
    split="train",
    download=True
)
trainset.initialize()


# initialize testing set from SpuCoMNIST
testset = SpuCoMNIST(
    root="data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes = classes,
    split="test",
    download=True
)
testset.initialize()

100%|██████████| 10000/10000 [00:00<00:00, 33976.14it/s]


In [6]:
# view a example image from training set
# after resizing to conventional 28,28
T.ToPILImage()(trainset[0][0]).resize((28,28))

In [7]:
# verify that images are 28x28 with 3 color channels
# expect [3,28,28]
trainset[0][0].shape

torch.Size([3, 28, 28])

In [8]:
from spuco.models import model_factory
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(DEVICE)


In [9]:
from torch.optim import SGD

# NOTES: noticed some issue with dump pickler when using multiple workers
# for DataLoaders in both ERM and Evaluator classes

# not completely sure the cause (why mp pickling was broken on my system) 
# setting num_workers=0 rectified the issue, with slightly slower data loading
erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer = SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=DEVICE,
    verbose=True,
)

In [10]:
erm.train()

Epoch 0: 100%|██████████| 751/751 [00:05<00:00, 129.09batch/s, accuracy=100.0%, loss=0.011]   


In [11]:
# robust accuracy eval

from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=DEVICE,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:  20%|██        | 5/25 [00:00<00:00, 44.53it/s]

Group (0, 0) Accuracy: 100.0
Group (0, 1) Accuracy: 0.0
Group (0, 2) Accuracy: 0.0
Group (0, 3) Accuracy: 0.0
Group (0, 4) Accuracy: 0.0
Group (1, 0) Accuracy: 0.0
Group (1, 1) Accuracy: 100.0
Group (1, 2) Accuracy: 0.0
Group (1, 3) Accuracy: 0.0


Evaluating group-wise accuracy:  60%|██████    | 15/25 [00:00<00:00, 40.37it/s]

Group (1, 4) Accuracy: 0.0
Group (2, 0) Accuracy: 0.0
Group (2, 1) Accuracy: 0.0
Group (2, 2) Accuracy: 100.0
Group (2, 3) Accuracy: 0.0
Group (2, 4) Accuracy: 0.0
Group (3, 0) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:00<00:00, 40.78it/s]

Group (3, 1) Accuracy: 0.0
Group (3, 2) Accuracy: 0.0
Group (3, 3) Accuracy: 100.0
Group (3, 4) Accuracy: 0.0
Group (4, 0) Accuracy: 0.0
Group (4, 1) Accuracy: 0.0
Group (4, 2) Accuracy: 0.0
Group (4, 3) Accuracy: 0.0
Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [12]:
evaluator.worst_group_accuracy

((0, 1), 0.0)

In [13]:
evaluator.average_accuracy

99.49379218398467

In [14]:
# we observe that although average accuracy over all groups is very high
# we see that within subgroups that accuracy is as low as 0

# --------------------------------------
# STEP 2: Cluster inputs into sublabels
# --------------------------------------
from spuco.group_inference import Cluster, ClusterAlg

logits = erm.trainer.get_trainset_outputs()
cluster = Cluster(
    Z=logits,
    class_labels=trainset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=2,
    device=DEVICE,
    verbose=True
)
# partition train dataset into clusters using unsupervised
# NOTE: ran into same mp num_workers DataLoader issue, set num_workers=0 within get_trainset_outputs

group_partition = cluster.infer_groups()

Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00, 11.77it/s]


In [15]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key])) # print inferred groups and their sizes

(0, 0) 10082
(0, 1) 51
(1, 0) 9623
(1, 1) 49
(2, 0) 8965
(2, 1) 46
(3, 0) 9698
(3, 1) 49
(4, 0) 9416
(4, 1) 25


In [16]:

# --------------------------------------
# STEP 3: retrain lenet using group balancing
# --------------------------------------

from spuco.robust_train import GroupBalanceBatchERM

balanced_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(DEVICE)

group_balance_erm = GroupBalanceBatchERM(
    model=model, # could also perform another epoch(s) on previously ERM'd model
    num_epochs=4,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(balanced_model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=DEVICE,
    verbose=True
)

group_balance_erm.train()

Epoch 3: 100%|██████████| 751/751 [00:06<00:00, 122.31batch/s, accuracy=100.0%, loss=0.0244] 


In [19]:
# evaluate group balanced model
balanced_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=balanced_model,
    device=DEVICE,
    verbose=True
)

balanced_evaluator.evaluate()

Evaluating group-wise accuracy:   0%|          | 0/25 [00:00<?, ?it/s]

Evaluating group-wise accuracy:  16%|█▌        | 4/25 [00:00<00:01, 15.16it/s]

Group (0, 0) Accuracy: 96.6903073286052
Group (0, 1) Accuracy: 57.4468085106383
Group (0, 2) Accuracy: 46.808510638297875
Group (0, 3) Accuracy: 55.082742316784866
Group (0, 4) Accuracy: 71.63120567375887
Group (1, 0) Accuracy: 74.32762836185819


Evaluating group-wise accuracy:  44%|████▍     | 11/25 [00:00<00:00, 26.34it/s]

Group (1, 1) Accuracy: 89.24205378973105
Group (1, 2) Accuracy: 72.30392156862744
Group (1, 3) Accuracy: 60.294117647058826
Group (1, 4) Accuracy: 62.5
Group (2, 0) Accuracy: 67.2
Group (2, 1) Accuracy: 41.86666666666667
Group (2, 2) Accuracy: 89.06666666666666
Group (2, 3) Accuracy: 35.46666666666667
Group (2, 4) Accuracy: 41.97860962566845


Evaluating group-wise accuracy:  80%|████████  | 20/25 [00:00<00:00, 34.42it/s]

Group (3, 0) Accuracy: 75.37688442211055
Group (3, 1) Accuracy: 70.27707808564232
Group (3, 2) Accuracy: 74.81108312342569
Group (3, 3) Accuracy: 97.48110831234257
Group (3, 4) Accuracy: 74.05541561712846
Group (4, 0) Accuracy: 27.455919395465994
Group (4, 1) Accuracy: 80.85642317380352
Group (4, 2) Accuracy: 71.78841309823677
Group (4, 3) Accuracy: 8.585858585858587


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:00<00:00, 30.30it/s]

Group (4, 4) Accuracy: 97.22222222222223


{(0, 0): 96.6903073286052,
 (0, 1): 57.4468085106383,
 (0, 2): 46.808510638297875,
 (0, 3): 55.082742316784866,
 (0, 4): 71.63120567375887,
 (1, 0): 74.32762836185819,
 (1, 1): 89.24205378973105,
 (1, 2): 72.30392156862744,
 (1, 3): 60.294117647058826,
 (1, 4): 62.5,
 (2, 0): 67.2,
 (2, 1): 41.86666666666667,
 (2, 2): 89.06666666666666,
 (2, 3): 35.46666666666667,
 (2, 4): 41.97860962566845,
 (3, 0): 75.37688442211055,
 (3, 1): 70.27707808564232,
 (3, 2): 74.81108312342569,
 (3, 3): 97.48110831234257,
 (3, 4): 74.05541561712846,
 (4, 0): 27.455919395465994,
 (4, 1): 80.85642317380352,
 (4, 2): 71.78841309823677,
 (4, 3): 8.585858585858587,
 (4, 4): 97.22222222222223}

In [20]:
balanced_evaluator.worst_group_accuracy # worst group accuracy has increased

((4, 3), 8.585858585858587)

In [21]:
balanced_evaluator.average_accuracy # average accuracy has dropped slightly

93.8467735913562